In [6]:
import os
import webdataset as wds
import logging
import numpy
import json
import torch
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from pathlib import Path
from typing import Optional
from itertools import islice
from glob import glob
from torch.utils.data import DataLoader, IterableDataset
from webdataset.filters import pipelinefilter
from webdataset.utils import pytorch_worker_info
import pickle
from traceback_with_variables import activate_in_ipython_by_import
from sklearn.preprocessing import StandardScaler

In [7]:
ds = wds.WebDataset('exp/local_v2/train_aed_6/val/validation-000000.tar').decode()

In [15]:
for e in ds:
    x, y, _ = e['predicted.pth'][-1].abs()
    print(x - e['x.pth'][-1].abs(), y - e['y.pth'][-1].abs())
    

tensor(-10.7339, dtype=torch.float64) tensor(2.7895, dtype=torch.float64)
tensor(0.4911, dtype=torch.float64) tensor(-1.8461, dtype=torch.float64)
tensor(5.3167, dtype=torch.float64) tensor(-3.3208, dtype=torch.float64)
tensor(11.1057, dtype=torch.float64) tensor(-4.3742, dtype=torch.float64)
tensor(-0.1032, dtype=torch.float64) tensor(-1.3734, dtype=torch.float64)
tensor(-3.1591, dtype=torch.float64) tensor(-0.0370, dtype=torch.float64)
tensor(-1.1877, dtype=torch.float64) tensor(0.2471, dtype=torch.float64)
tensor(-0.5150, dtype=torch.float64) tensor(-1.0370, dtype=torch.float64)
tensor(-0.2579, dtype=torch.float64) tensor(-0.6066, dtype=torch.float64)
tensor(-0.0809, dtype=torch.float64) tensor(-1.3813, dtype=torch.float64)
tensor(-0.4705, dtype=torch.float64) tensor(-3.5651, dtype=torch.float64)
tensor(-1.4517, dtype=torch.float64) tensor(-7.6350, dtype=torch.float64)
tensor(-4.3524, dtype=torch.float64) tensor(-9.1537, dtype=torch.float64)
tensor(-9.3922, dtype=torch.float64) tens

In [25]:
diffx = []
diffy = []
for e in ds:
    x, y, _ = e['predicted.pth'][-1]
    diffx.append(x/e['x.pth'][-1])
    diffy.append(y/e['y.pth'][-1])
    print(diffx[-1], diffy[-1])

tensor(0.9974, dtype=torch.float64) tensor(1.0002, dtype=torch.float64)
tensor(1.0001, dtype=torch.float64) tensor(0.9999, dtype=torch.float64)
tensor(1.0013, dtype=torch.float64) tensor(0.9998, dtype=torch.float64)
tensor(1.0028, dtype=torch.float64) tensor(0.9997, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64) tensor(0.9999, dtype=torch.float64)
tensor(0.9992, dtype=torch.float64) tensor(1.0000, dtype=torch.float64)
tensor(0.9997, dtype=torch.float64) tensor(1.0000, dtype=torch.float64)
tensor(0.9999, dtype=torch.float64) tensor(0.9999, dtype=torch.float64)
tensor(0.9999, dtype=torch.float64) tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64) tensor(0.9999, dtype=torch.float64)
tensor(0.9999, dtype=torch.float64) tensor(0.9989, dtype=torch.float64)
tensor(0.9998, dtype=torch.float64) tensor(0.9977, dtype=torch.float64)
tensor(0.9993, dtype=torch.float64) tensor(0.9972, dtype=torch.float64)
tensor(0.9986, dtype=torch.float64) tensor(0.9996, dtype=torch.f

In [27]:
torch.mean(torch.asarray(diffx)), torch.mean(torch.asarray(diffy))

(tensor(0.9956, dtype=torch.float64), tensor(0.9956, dtype=torch.float64))

In [28]:
for e in ds:
    x, y, _ = e['predicted.pth'][-1]
    print(x/e['x.pth'][-1] /0.9956 , y/e['y.pth'][-1] / 0.9956)

tensor(1.0018, dtype=torch.float64) tensor(1.0046, dtype=torch.float64)
tensor(1.0045, dtype=torch.float64) tensor(1.0043, dtype=torch.float64)
tensor(1.0057, dtype=torch.float64) tensor(1.0042, dtype=torch.float64)
tensor(1.0072, dtype=torch.float64) tensor(1.0041, dtype=torch.float64)
tensor(1.0044, dtype=torch.float64) tensor(1.0043, dtype=torch.float64)
tensor(1.0036, dtype=torch.float64) tensor(1.0044, dtype=torch.float64)
tensor(1.0041, dtype=torch.float64) tensor(1.0044, dtype=torch.float64)
tensor(1.0043, dtype=torch.float64) tensor(1.0044, dtype=torch.float64)
tensor(1.0044, dtype=torch.float64) tensor(1.0044, dtype=torch.float64)
tensor(1.0044, dtype=torch.float64) tensor(1.0043, dtype=torch.float64)
tensor(1.0043, dtype=torch.float64) tensor(1.0034, dtype=torch.float64)
tensor(1.0042, dtype=torch.float64) tensor(1.0021, dtype=torch.float64)
tensor(1.0038, dtype=torch.float64) tensor(1.0016, dtype=torch.float64)
tensor(1.0030, dtype=torch.float64) tensor(1.0040, dtype=torch.f

In [17]:
e['predicted.pth'].shape

torch.Size([1000, 3])

In [12]:
e.keys()

dict_keys(['__key__', '__url__', 'control_feats.pth', '__local_path__', 'loc.pth', 'metric.npy', 'metrics.npy', 'predicted.pth', 'start_ns.id', 'x.pth', 'y.pth', 'yaw.pth'])

In [10]:
e['predicted.pth']

tensor([[-1.6560e+03, -1.0550e+03,  6.2696e-01],
        [-1.6559e+03, -1.0550e+03,  6.2696e-01],
        [-1.6559e+03, -1.0550e+03,  6.3064e-01],
        ...,
        [-1.6414e+03, -1.0453e+03,  6.4180e-01],
        [-1.6414e+03, -1.0452e+03,  4.9232e-01],
        [-1.6414e+03, -1.0452e+03,  4.9232e-01]], dtype=torch.float64)

In [14]:
torch.nn.functional.pad(torch.ones(2,5), (0,0, 1, 2))

tensor([[0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [22]:
a = torch.nn.functional.pad(
    torch.ones(2, 5), (0, 10,), #mode="replicate"         
)   
a.shape

torch.Size([2, 15])

In [12]:
e['predicted_speed_loc.pth']

tensor([[0.0430, 0.0314],
        [0.0434, 0.0313],
        [0.0433, 0.0313],
        ...,
        [0.0649, 0.0419],
        [0.0649, 0.0419],
        [0.0646, 0.0421]])

In [13]:
e['__local_path__']

'exp/local_v2/train_speed_mse_1/validation-000000.tar'

In [14]:
e['x.pth']

tensor([-1655.9896, -1655.9459, -1655.9022, -1655.8588, -1655.8155, -1655.7723,
        -1655.7290, -1655.6848, -1655.6406, -1655.5977, -1655.5547, -1655.5113,
        -1655.4680, -1655.4254, -1655.3829, -1655.3406, -1655.2983, -1655.2560,
        -1655.2137, -1655.1711, -1655.1284, -1655.0861, -1655.0437, -1655.0018,
        -1654.9600, -1654.9183, -1654.8766, -1654.8342, -1654.7918, -1654.7505,
        -1654.7091, -1654.6677, -1654.6263, -1654.5853, -1654.5442, -1654.5033,
        -1654.4624, -1654.4204, -1654.3784, -1654.3378, -1654.2973, -1654.2567,
        -1654.2162, -1654.1761, -1654.1359, -1654.0958, -1654.0558, -1654.0149,
        -1653.9741, -1653.9343, -1653.8944, -1653.8546, -1653.8147, -1653.7753,
        -1653.7360, -1653.6969, -1653.6578, -1653.6178, -1653.5779, -1653.5389,
        -1653.5000, -1653.4611, -1653.4223, -1653.3836, -1653.3449, -1653.3064,
        -1653.2679, -1653.2296, -1653.1912, -1653.1522, -1653.1132, -1653.0754,
        -1653.0376, -1652.9996, -1652.96

In [15]:
e['y.pth']

tensor([-1055.0446, -1055.0129, -1054.9813, -1054.9494, -1054.9175, -1054.8858,
        -1054.8540, -1054.8226, -1054.7912, -1054.7598, -1054.7284, -1054.6971,
        -1054.6658, -1054.6348, -1054.6038, -1054.5731, -1054.5424, -1054.5119,
        -1054.4814, -1054.4506, -1054.4198, -1054.3892, -1054.3586, -1054.3284,
        -1054.2982, -1054.2682, -1054.2382, -1054.2080, -1054.1777, -1054.1481,
        -1054.1185, -1054.0892, -1054.0599, -1054.0305, -1054.0012, -1053.9720,
        -1053.9429, -1053.9137, -1053.8844, -1053.8557, -1053.8269, -1053.7984,
        -1053.7700, -1053.7416, -1053.7132, -1053.6849, -1053.6566, -1053.6279,
        -1053.5993, -1053.5713, -1053.5432, -1053.5156, -1053.4880, -1053.4603,
        -1053.4327, -1053.4053, -1053.3780, -1053.3504, -1053.3228, -1053.2956,
        -1053.2684, -1053.2414, -1053.2144, -1053.1874, -1053.1605, -1053.1338,
        -1053.1070, -1053.0804, -1053.0538, -1053.0275, -1053.0011, -1052.9750,
        -1052.9488, -1052.9225, -1052.89

In [16]:
e['predicted.pth'].shape

torch.Size([1000, 3])

In [18]:
torch.abs(e['predicted.pth'][:, 0] - e['x.pth'])

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [21]:
diff = torch.stack([torch.abs(e['predicted.pth'][:, 0] - e['x.pth']), torch.abs(e['loc.pth'][:, 0])], dim=-1)
diff = diff[(diff!=0).all(dim=-1)]
print(diff)


tensor([[ 0.4071,  0.0453],
        [ 0.4075,  0.0455],
        [ 0.4080,  0.0455],
        ...,
        [41.5835,  0.0649],
        [41.6026,  0.0649],
        [41.6357,  0.0646]], dtype=torch.float64)


In [32]:
diff = torch.stack([e['x.pth'][1:] - e['x.pth'][:-1], e['loc.pth'][1:, 0]], dim=-1)
#diff = diff[(diff!=0).all(dim=-1)]
print(diff.sum())


tensor(112.5078, dtype=torch.float64)


In [31]:
diff[:, 0] - diff[:, 1]

tensor([ 1.0778e-09, -1.5948e-09, -1.5784e-09, -1.5784e-09,  3.5561e-10,
         4.4929e-10,  6.5575e-10, -3.4584e-10, -3.8381e-10,  8.8653e-10,
         1.7631e-09,  9.5770e-10,  7.2851e-10,  1.6894e-10, -1.1376e-09,
        -1.1378e-09,  2.4784e-10,  1.4338e-09,  1.2012e-09, -1.1353e-09,
        -7.5079e-10, -4.3838e-10, -5.7094e-10, -8.2377e-10, -5.2751e-10,
        -5.2796e-10, -1.5577e-09, -1.7144e-09, -6.7007e-10, -1.7212e-09,
         7.6625e-10,  7.6648e-10,  5.7662e-10, -2.1146e-11,  9.1404e-10,
         1.4700e-09,  4.2155e-10,  5.0204e-10, -6.2460e-10, -6.2437e-10,
         7.5124e-10, -5.2296e-10,  9.3451e-11,  9.3451e-11, -1.2133e-09,
         1.5639e-09, -9.1791e-10, -9.1791e-10,  5.3114e-10, -7.3510e-10,
         1.8169e-09, -9.8476e-10, -3.9495e-10,  1.5530e-10, -1.2169e-09,
         7.4283e-10,  3.6925e-10,  3.6948e-10, -1.5461e-11,  9.7134e-10,
         1.4425e-09,  6.6120e-10, -1.3013e-09, -1.3010e-09, -1.2833e-09,
         1.7633e-09, -7.8580e-10,  1.3886e-09, -1.7